**Machine Learning for CV - Project**

In [ ]:
# Install torch https://pytorch.org/get-started/locally/

%pip install numpy kagglehub matplotlib opencv-python --quiet
%pip install segmentation-models-pytorch --quiet

*Import libraries*

In [ ]:
import os
import csv
import cv2
import kagglehub
import numpy as np

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils
from segmentation_models_pytorch.encoders import get_preprocessing_fn

from torch.utils.data import Dataset as BaseDataset

# Semantic Segmentation of Underwater Imagery

Dataset: https://www.kaggle.com/datasets/ashish2001/semantic-segmentation-of-underwater-imagery-suim

In [ ]:
path = kagglehub.dataset_download("ashish2001/semantic-segmentation-of-underwater-imagery-suim")
print("Path to dataset files:", path)

### Data preprocessing

In [4]:
# Mapping categories in TEST/masks
category_colors = {
    "Saliency": [0, 0, 0],  # Background (waterbody) (Called also BW)
    "HD": [0, 0, 255],      # Human divers
    "PF": [0, 255, 0],      # Aquatic plants and sea-grass
    "WR": [0, 255, 255],    # Wrecks and ruins
    "RO": [255, 0, 0],      # Robots (AUVs/ROVs/instruments)
    "RI": [255, 0, 255],    # Reefs and invertebrates
    "FV": [255, 255, 0],    # Fish and vertebrates
    "SR": [255, 255, 255],  # Sea-floor and rocks
}

CLASSES = list(category_colors.keys())
color_to_class = {tuple(value): idx for idx, (_, value) in enumerate(category_colors.items())}
idx_to_class = {idx: class_name for idx, class_name in enumerate(CLASSES)}

In [5]:
def convert_rgb_to_class(mask, color_to_class):
    h, w, _ = mask.shape
    class_mask = np.zeros((h, w), dtype=np.int64)
    
    for color, class_idx in color_to_class.items():
        color = np.array(color, dtype=np.uint8)
        matches = np.all(mask == color, axis=-1)  # Find pixels matching this color
        class_mask[matches] = class_idx
    
    return class_mask

In [6]:
class CustomDataset(BaseDataset):
    def __init__(self, base_path, train=True, augmentation=None, preprocessing=None):        
        self.base_path = base_path
        self.train = train
        self.augmentation = augmentation
        self.preprocessing = preprocessing

        if train:
            # Training/validation data
            self.images_dir = os.path.join(base_path, "train_val", "images")
            self.masks_dir = os.path.join(base_path, "train_val", "masks")
        else:
            # Test data
            self.images_dir = os.path.join(base_path, "TEST", "images")
            self.masks_dir = os.path.join(base_path, "TEST", "masks")
        
        self.image_files = os.listdir(self.images_dir)
        self.class_values = [key for key,_ in enumerate(CLASSES)]

        # Get all the mask files from the main masks folder and its subdirectories (for test set)
        if train:
            self.mask_files = os.listdir(self.masks_dir)
            self.subfolders = None
        else:
            self.mask_files = [
                filename for filename in os.listdir(self.masks_dir)
                if os.path.isfile(os.path.join(self.masks_dir, filename))
            ]
            self.subfolders = [
                filename for filename in os.listdir(self.masks_dir)
                if os.path.isdir(os.path.join(self.masks_dir, filename))
            ]

    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image with cv2
        img_path = os.path.join(self.images_dir, self.image_files[idx])
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        mask_name = self.image_files[idx].split(".")[0]+".bmp"

        # Prepare mask
        mask_path = os.path.join(self.masks_dir, mask_name)
        rgb_mask = cv2.imread(mask_path, cv2.IMREAD_COLOR)
        rgb_mask = cv2.cvtColor(rgb_mask, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        # Convert RGB mask to class index mask
        mask = convert_rgb_to_class(rgb_mask, color_to_class)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
                       
        return image, mask
    
    def get_mask_by_class(self, item_idx, class_name):
        if self.train == True:
            return None
        else:
            if isinstance(class_name, int):
                class_name = idx_to_class[class_name]
            mask_name = self.image_files[item_idx].split(".")[0]+".bmp"
            mask_path = os.path.join(self.masks_dir, class_name, mask_name)
            rgb_mask = cv2.imread(mask_path, cv2.IMREAD_COLOR)
            rgb_mask = cv2.cvtColor(rgb_mask, cv2.COLOR_BGR2RGB) 
            return rgb_mask

In [7]:
def visualize(image, mask, alpha=0.5, save=None):
    # Normalize the image if necessary
    if image.max() > 1:
        image = image / 255.0

    # Determine if the mask is multi-class or RGB
    if mask.ndim == 3 and mask.shape[-1] > 3:  # Multi-class segmentation
        # Define a color map with unique colors for each class
        colors = plt.get_cmap('tab20', mask.shape[2])
        cmap = ListedColormap([colors(i) for i in range(mask.shape[2])])

        # Get the class with the highest score at each pixel
        combined_mask = np.argmax(mask, axis=2)

        # Create a color image of the mask for overlay
        color_mask = cmap(combined_mask / combined_mask.max())[:, :, :3]  # Remove alpha channel if present
    elif mask.ndim == 3 and mask.shape[-1] == 3:  # RGB image mask
        # Normalize RGB mask if necessary
        if mask.max() > 1:
            mask = mask / 255.0
        color_mask = mask
    else:
        raise ValueError("Mask must have shape (H, W, num_classes) or (H, W, 3).")

    # Ensure the image has the shape (height, width, channels)
    if image.shape[0] == 3:  # If shape is (3, height, width)
        image = np.transpose(image, (1, 2, 0))  # Transpose to (height, width, 3)

    # Overlay the mask on the original image
    overlayed_image = (1 - alpha) * image + alpha * color_mask

    # Clip values to ensure they stay within [0, 1]
    overlayed_image = np.clip(overlayed_image, 0, 1)

    # Plot original and overlayed images side by side
    plt.figure(figsize=(12, 6))

    # Show the original image
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    # Show the image with segmentation overlay
    plt.subplot(1, 2, 2)
    plt.imshow(overlayed_image)
    plt.title("Image with Segmentation Overlay")
    plt.axis('off')
    if save is not None:
        plt.savefig(f"{save}.png")
    plt.show()

In [8]:
def create_folder_if_not_exist(folder):
  if not os.path.exists(folder):
      os.makedirs(folder)
      
def write_csv(filename, headers, values, multi_rows=False):
  with open(f"{filename}.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        if multi_rows == True:
          for v in values:
            writer.writerow(v)
        else:
          writer.writerow(values)

In [9]:
def check_pad(image, multiple=32):
  h, w = image.shape[:2]
  return h % multiple==0 and w % multiple==0

def pad_to_multiple(image, multiple=32):
    """
    Pad an image so that its height and width are divisible by a given multiple.
    Args:
        image: Input image as a numpy array.
        multiple: The number to which height and width should be divisible.

    Returns:
        Padded image.
    """
    h, w = image.shape[:2]
    
    # Calculate the necessary padding
    new_h = ((h + multiple - 1) // multiple) * multiple
    new_w = ((w + multiple - 1) // multiple) * multiple
    
    pad_h = new_h - h
    pad_w = new_w - w
    
    # Pad the image (adding padding evenly to top, bottom, left, right)
    padded_image = cv2.copyMakeBorder(
        image,
        0, pad_h,   # Top and bottom padding
        0, pad_w,   # Left and right padding
        cv2.BORDER_CONSTANT,
        value=[0, 0, 0]  # Black padding
    )
    return padded_image

## 4. Testing the model

In [10]:
MODEL_NAME = "Unet-resnet50"
EPOCHS = 100
MODEL_FOLDER = f"./results/{MODEL_NAME}"
INDEX_RUN = 2
  
RESULTS_FOLDER = f"{MODEL_FOLDER}/{INDEX_RUN}-epochs{EPOCHS}"

MASKS_FOLDER = f"{RESULTS_FOLDER}/masks"
TEST_FOLDER = f"{RESULTS_FOLDER}/test"
create_folder_if_not_exist(TEST_FOLDER)

BEST_MODEL = f'{RESULTS_FOLDER}/best_model.pth'

In [ ]:
ENCODER = 'resnet18'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'softmax2d'  # Use sigmoid  if doing one-single-class segmentation
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class UNetWithDropout(smp.Unet):
    def __init__(self, encoder_name, encoder_weights, classes, activation, in_channels=3, dropout_prob=0.5):
        super().__init__(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            classes=classes,
            activation=activation,
            in_channels=in_channels
        )
        self.dropout = nn.Dropout2d(p=dropout_prob)  # Dropout layer

    def forward(self, x):
        """Forward method with dropout added to the encoder and decoder outputs."""
        features = self.encoder(x)
        features = [self.dropout(feature) for feature in features]  # Apply dropout after each encoder layer
        decoder_output = self.decoder(*features)  # Decode features
        masks = self.segmentation_head(decoder_output)  # Generate segmentation mask
        return masks

# Create FPN model with pretrained encoder
model = UNetWithDropout(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,  # Use 'imagenet' pretrained weights for encoder initialization
    classes=len(CLASSES),  # Number of classes in your dataset
    activation=ACTIVATION,  # Activation function for the output
    in_channels=3,  # Model input channels (1 for gray-scale images, 3 for RGB, etc.)
    dropout_prob=0.3
    # decoder_dropout=0.5
)

# Get preprocessing function for the encoder
preprocessing_fn = get_preprocessing_fn(ENCODER, pretrained=ENCODER_WEIGHTS)

print("Running on: ", DEVICE)

In [ ]:
# Class pixel counts from your data
pixel_counts = torch.tensor([189688979, 12313599, 11940788, 37353994, 2917692, 186100654, 39309599, 78415339])

# Compute weights inversely proportional to pixel counts
total_pixels = pixel_counts.sum()
class_weights = total_pixels / (pixel_counts * len(pixel_counts))
class_weights += class_weights.mean()
class_weights = class_weights / class_weights.max()  # Normalize weights to the range [0, 1]
print("Normalized Class Weights:", class_weights)

In [13]:
class WeightedDiceLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedDiceLoss, self).__init__()
        self.class_weights = class_weights
        self.__name__ = "dice_loss"  # Add this line
        
    def forward(self, outputs, targets):
        dice_loss = 0.0
        for class_idx, weight in enumerate(self.class_weights):
            # Check if the class is present in either targets or outputs
            if targets[:, class_idx].sum() > 0 or outputs[:, class_idx].sum() > 0:
                # Calculate Dice for this class
                intersection = (outputs[:, class_idx] * targets[:, class_idx]).sum()
                union = outputs[:, class_idx].sum() + targets[:, class_idx].sum()
                dice = (2.0 * intersection) / (union + 1e-5)

                # Apply the weight to the loss for this class
                dice_loss += weight * (1 - dice)

        return dice_loss / len(self.class_weights)

class_weights = class_weights.to(DEVICE)  # Move to the correct device

weighted_loss = WeightedDiceLoss(class_weights)

In [14]:
loss = weighted_loss #utils.losses.DiceLoss() 
metrics = [
    utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([
    dict(params=model.parameters(), lr=0.0001, weight_decay=1e-4),
])

In [ ]:
best_model = torch.load(BEST_MODEL)

### Visualization

Let's some outputs of the model on test data

In [16]:
test_dataset = CustomDataset(base_path=path, train=False)

In [ ]:
base_headers = ["Pixel predicted", "Pixel true", "Pixel intersetion", "Pixel union"]
headers = ["Image"] + [f"{value} {idx}" for value in base_headers for idx in idx_to_class]
headers

In [18]:
class_colors = {v: k for k, v in color_to_class.items()}

def create_color_mask(pred_mask, class_colors):
    # Initialize the colored mask with black (default background)
    height, width = pred_mask.shape
    colored_mask = np.zeros((height, width, 3), dtype=np.uint8)
    
    masks_per_class = []

    # Loop through each class and apply the corresponding color
    for class_id, color in class_colors.items():
        class_mask = (pred_mask == class_id)  # Create a binary mask for the class
        colored_mask[class_mask] = color      # Apply the color where the class mask is 1
        
        mask_per_class = np.full((height, width, 3), fill_value=(200, 200, 200), dtype=np.uint8) # Set a light grey color to see also the black mask
        mask_per_class[class_mask] = color
        masks_per_class.append(mask_per_class)

    return colored_mask, masks_per_class

In [ ]:
csv_file = f'{TEST_FOLDER}/results.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()  # Write the header once
    
    for i in range(len(test_dataset)):# [15, 34, 55, 87, 64, 77, 95]:#range(5):
        # Randomly select an image from the test dataset
        # n = np.random.choice(len(test_dataset))
        n=i
        image_vis = test_dataset[n][0].astype('uint8')  # Original image for visualization
        image, gt_mask = test_dataset[n]  # Image and ground truth
        
        
        if not check_pad(image):
            image = pad_to_multiple(image)
            image_vis = pad_to_multiple(image_vis)
            gt_mask = pad_to_multiple(gt_mask)

        gt_mask = gt_mask.squeeze()  # Squeeze the ground truth mask

        # Transpose image to match model input shape (C, H, W)
        image = np.transpose(image, (2, 0, 1))  # Convert (H, W, C) -> (C, H, W)
        
        # Convert to tensor and move to the appropriate device
        x_tensor = torch.from_numpy(image).float().to(DEVICE).unsqueeze(0)  # Add batch dimension

        # Perform prediction
        with torch.no_grad():
            pr_mask = best_model(x_tensor)  # Predict mask
            # pr_mask = (pr_mask.squeeze().cpu().numpy().round())  # Convert to numpy array and round values
            pr_mask = torch.argmax(pr_mask, dim=1).squeeze(0).cpu().numpy()
                
        colored_mask, colored_mask_per_class = create_color_mask(pr_mask, class_colors)
        print(n)
        # Visualize the input image and predicted mask
        visualize(
            image=image_vis,
            mask=colored_mask,
            alpha=0.5,
            save=f"{TEST_FOLDER}/Predicted segmentation image {n}"
        )
        visualize(
            image=image_vis,
            mask=gt_mask,
            alpha=0.7
        )
        
        for idx in range(len(CLASSES)):
            class_masks = test_dataset.get_mask_by_class(n, idx)
            
            # plt.subplot(1, 2, 1)
            # plt.imshow(colored_mask_per_class[idx])  # Show the color-coded mask
            # plt.title(f'Generated class mask: {idx_to_class[idx]}')
            # plt.axis('off')

            # plt.subplot(1, 2, 2)
            # plt.imshow(class_masks)  # Show the color-coded mask
            # plt.title(f'Original class mask: {idx_to_class[idx]}')
            
            # plt.axis('off')
            # plt.savefig(f"{RESULTS_FOLDER}/masks/{n}class{idx}.png")

            # plt.show()
            
            # Initialize a dictionary to hold row data for the CSV
        row = {'Image': n}

        for class_id in range(len(CLASSES)):
            class_masks = test_dataset.get_mask_by_class(n, idx_to_class[class_id])
            # Create binary masks for ground truth and prediction
            pred_mask = colored_mask_per_class[class_id]
            true_mask = class_masks
            min_height = min(pred_mask.shape[0], true_mask.shape[0])
            min_width = min(pred_mask.shape[1], true_mask.shape[1])

            # Crop both masks to the smallest shape
            pred_mask = pred_mask[:min_height, :min_width]
            true_mask = true_mask[:min_height, :min_width]
            
            pred_mask_binary = np.any(pred_mask != (200, 200, 200), axis=-1).astype(np.uint8)
            true_mask_binary = np.all(true_mask == (255, 255, 255), axis=-1).astype(np.uint8)
            
            # Calculate intersection and union
            intersection = np.sum(np.logical_and(pred_mask_binary, true_mask_binary))
            union = np.sum(np.logical_or(pred_mask_binary, true_mask_binary))

            # Calculate the pixel counts for each mask
            predicted_pixels = np.sum(pred_mask_binary)
            true_pixels = np.sum(true_mask_binary)

            # Populate the CSV row
            row[f'Pixel predicted {class_id}'] = predicted_pixels
            row[f'Pixel true {class_id}'] = true_pixels
            row[f'Pixel intersetion {class_id}'] = intersection
            row[f'Pixel union {class_id}'] = union

        # Write the row directly to the CSV file
        writer.writerow(row)
        
        plt.imshow(colored_mask)  # Show the color-coded mask
        plt.title('Color-coded Mask')
        plt.axis('off')
        plt.savefig(f"{RESULTS_FOLDER}/masks/{n}full_mask.png")
        # plt.show()
        

    print(f"Results saved to {csv_file}")

In [ ]:
%pip install pandas matplotlib seaborn numpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data from the CSV
csv_file = f'{TEST_FOLDER}/results.csv'
df = pd.read_csv(csv_file)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve

# Assuming df is your dataframe
# Extracting predicted and true pixel values for plot generation
pixel_columns_predicted = [f'Pixel predicted {i}' for i in range(8)]
pixel_columns_true = [f'Pixel true {i}' for i in range(8)]
pixel_columns_predicted_renamed = [f'Class {idx_to_class[i]}' for i in range(8)]

# 1. Heatmap: Predicted vs True Labels (for the first 8 pixel categories)
# Create heatmap for the first image in the dataset
heatmap_data = df.iloc[0][pixel_columns_predicted].values.reshape(1, -1)
true_labels_data = df.iloc[0][pixel_columns_true].values.reshape(1, -1)

plt.figure(figsize=(10, 3))
sns.heatmap([heatmap_data[0], true_labels_data[0]], annot=True, cmap="Blues", xticklabels=pixel_columns_predicted_renamed, yticklabels=['Predicted', 'True'])
plt.title(f"Image 0: Pixel Predictions vs True Labels")
plt.xlabel("Pixel Categories")
plt.ylabel("Labels")
plt.show()

# 2. Precision-Recall for Pixel-wise Classification (Pixel intersection)
# Calculate Precision-Recall curve for 'intersection' pixels
pixel_columns_intersection = [f'Pixel intersetion {i}' for i in range(8)]
true_intersection = df[pixel_columns_true].values.ravel()
predicted_intersection = df[pixel_columns_intersection].values.ravel()
from sklearn.metrics import precision_recall_curve
import numpy as np
import matplotlib.pyplot as plt

# Initialize dictionaries to store precision and recall for each class
precision_dict = {}
recall_dict = {}

# Loop over each class (0 to 7)
for i in range(8):
    # Convert true and predicted values for current class (One-vs-Rest)
    true_values = (df[f'Pixel true {i}'].values > 0).astype(int)  # 1 if the class is true, otherwise 0
    predicted_values = (df[f'Pixel predicted {i}'].values > 0).astype(int)  # 1 if predicted, otherwise 0
    
    # Compute the precision and recall for the current class
    precision, recall, _ = precision_recall_curve(true_values, predicted_values)
    
    # Store results in the dictionary
    precision_dict[i] = precision
    recall_dict[i] = recall

# Plot Precision-Recall curves for each class
plt.figure(figsize=(10, 8))
for i in range(8):
    plt.plot(recall_dict[i], precision_dict[i], label=f'Pixel Class {i}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves for Each Pixel Class (One-vs-Rest)')
plt.legend(title='Pixel Classes')
plt.grid(True)
plt.show()


# Sum up all predicted and true values across the dataset
heatmap_data_sum = df[pixel_columns_predicted].sum().values.reshape(1, -1)
true_labels_data_sum = df[pixel_columns_true].sum().values.reshape(1, -1)

# Normalization of values
max_value = max([heatmap_data_sum.max(), true_labels_data_sum.max()])
heatmap_data_sum = heatmap_data_sum / max_value
true_labels_data_sum = true_labels_data_sum / max_value

# Create a combined heatmap
plt.figure(figsize=(10, 3))
sns.heatmap(
    [heatmap_data_sum[0], true_labels_data_sum[0]],
    annot=True,
    fmt=".5f",
    cmap="Blues",
    xticklabels=pixel_columns_predicted_renamed,
    yticklabels=['Predicted', 'True']
)
plt.title("Summed Pixel Predictions vs True Labels (Normalized)")
plt.xlabel("Pixel Categories")
plt.ylabel("Labels")
plt.savefig(f"{TEST_FOLDER}/heat_map.png")
plt.show()

print(heatmap_data_sum)
print(true_labels_data_sum)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Models and corresponding file paths
models = ["DeepLabV3", "DeepLabV3Plus", "FPN", "Linknet", "MAnet", "Unet", "UnetPlusPlus"]
csv_files = [f"results/{val}/1-epochs10/training_results_lr=1e-4.csv" for val in models]

# Initialize a dictionary to store the data
data = {}

# Load the CSV files into a dictionary
for model, file in zip(models, csv_files):
    data[model] = pd.read_csv(file)

# Define the metrics for comparison
metrics = ["Train Loss", "Train IoU", "Validation Loss", "Validation IoU"]

# Plot each metric
for metric in metrics:
    plt.figure(figsize=(10, 6))
    for model in models:
        plt.plot(data[model][metric], label=model)
    plt.title(f"Comparison of {metric} Across Models")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True)
    plt.show()
    
data


In [ ]:
from matplotlib.colors import Normalize, LinearSegmentedColormap
from matplotlib.cm import get_cmap

test_csv_files = [f"results/{val}/1-epochs10/test_results_lr=1e-4.csv" for val in models]

test_loss = []
test_iou = []

# Load the test metrics
for file in test_csv_files:
    df = pd.read_csv(file)
    test_loss.append(df["Test Loss"].values[0])
    test_iou.append(df["Test IoU"].values[0])

# Normalize the values for coloring
loss_norm = Normalize(vmin=min(test_loss), vmax=max(test_loss))
iou_norm = Normalize(vmin=min(test_iou), vmax=max(test_iou))

# Get the colormap
cmap = get_cmap("RdYlGn")
greens = LinearSegmentedColormap.from_list("greens", ["#e6ffe6", "#006600"])  # Light green to dark green
reds = LinearSegmentedColormap.from_list("reds", ["#ffe6e6", "#990000"])     # Light red to dark red

# Create bar plots for Test Loss and Test IoU
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Test Loss Bar Plot
loss_colors = [reds(1 - loss_norm(val)) for val in test_loss]  # Invert the colormap
axes[0].bar(models, test_loss, color=loss_colors)
axes[0].set_title("Test Loss Across Models")
axes[0].set_ylabel("Test Loss")
axes[0].set_xticklabels(models, rotation=45)
axes[0].set_ylim(0,1)

# Test IoU Bar Plot
iou_colors = [greens(iou_norm(val)) for val in test_iou]
axes[1].bar(models, test_iou, color=iou_colors)
axes[1].set_title("Test IoU Across Models")
axes[1].set_ylabel("Test IoU")
axes[1].set_xticklabels(models, rotation=45)
axes[1].set_ylim(0,1)

# Add colorbars for context
fig.colorbar(plt.cm.ScalarMappable(norm=loss_norm, cmap=reds.reversed()), ax=axes[0], orientation='vertical', label='Loss Intensity')
fig.colorbar(plt.cm.ScalarMappable(norm=iou_norm, cmap=greens), ax=axes[1], orientation='vertical', label='IoU Intensity')
# Adjust layout and show plots
plt.tight_layout()
plt.show()

print(test_iou)
print(test_loss)

: 